In [ ]:
!docker run -p "6333:6333" -p "6334:6334" --name "dsa4265-qdrant" --rm -d qdrant/qdrant:latest

3f36da75e8d440baf9e3923d513606941e5a2f7d4901706946ad3d12419f4bd5


In [49]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333", prefer_grpc=True)
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='summarizer')])

In [74]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct
from sentence_transformers import SentenceTransformer
import os
file_path = '/home/sarahgohrazer/Projects/Stocks-MultiAgent/aapl_10k_forms/apple_filings_text/000032019324000123-aapl-20240928.txt' 
with open(file_path, 'r') as file:
    text = file.read()

# Divide the text into chunks
chunk_size = 3000  # Adjust this size as needed
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Initialize the sentence transformer model for embedding
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can use a different model if needed

# Generate embeddings for the chunks
embeddings = model.encode(chunks)


In [96]:
from qdrant_client import models, QdrantClient
if "summarizer_test" not in client.get_collections():
    client.recreate_collection(
        collection_name="summarizer_test",
        vectors_config=VectorParams(
        size=model.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)


# Prepare the points to upload to Qdrant
points = [
    PointStruct(
        id=idx,  # Use the index as the unique ID
        vector=model.encode([chunk])[0].tolist(),  # Convert the chunk to a vector
        payload={"text": chunk}  # Attach the chunk as the payload
    )
    for idx, chunk in enumerate(chunks)
]

# Upload points to Qdrant
client.upload_points(
    collection_name="summarizer_test",  # Replace with your collection name
    points=points
)

print("Text chunks have been uploaded to Qdrant successfully.")

/tmp/ipykernel_26794/2806727580.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Text chunks have been uploaded to Qdrant successfully.


In [98]:
# Let's now search for something

hits = client.query_points(
    collection_name="summarizer_test",
    query_vector=model.encode("what is the ticker symbol of the company?").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

AssertionError: Unknown arguments: ['query_vector']

In [69]:
client.set_model(client.DEFAULT_EMBEDDING_MODEL, providers=["CUDAExecutionProvider", "CPUExecutionProvider"])

In [65]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
def chunk_document(text, max_tokens=15000):
    # Split the document into sentences (or paragraphs) as needed
    chunks = []
    current_chunk = ""
    
    for sentence in text.split('. '):  # Simple sentence-based split (adjust as needed)
        if len(current_chunk.split()) + len(sentence.split()) <= max_tokens:
            current_chunk += sentence + '. '
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + '. '
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

def read_text_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

file_path = '/home/sarahgohrazer/Projects/Stocks-MultiAgent/aapl_10k_forms/apple_filings_text/000032019324000123-aapl-20240928.txt' 
text_apple_2024_10k = read_text_from_file(file_path)
chunks = chunk_document(text_apple_2024_10k)

# Generate embeddings for each chunk and store them in Qdrant
embeddings = [model.encode(chunk) for chunk in chunks]

In [ ]:
print(text_apple_2024_10k)  # Print the first 1000 characters of the text

In [70]:
client.add(
    collection_name = "summarizer",
    documents = [text_apple_2024_10k]
)

['1de3a23fc88e485fb571a38efaa5d429']

In [38]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai.embeddings_utils import get_embedding, cosine_similarity

In [87]:
prompt = """"
what is the ticker symbol of the company?
"""

In [99]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

# Retrieve the API key from the environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client_gpt = OpenAI(
  api_key=OPENAI_API_KEY,  # this is also the default, it can be omitted
)

In [51]:
from openai import AsyncOpenAI

client_gpt = AsyncOpenAI()
completion = await client_gpt.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
response = completion.choices[0].message.content
print(response)

I'm sorry but without knowing the company you are referring to, I cannot provide you with the ticker symbol. Please provide me with the name of the company so I can assist you further.


In [92]:
# Generate the query embedding using the same model
query_vector = model.encode([prompt])[0]  # Generate embedding for the query

# Query Qdrant using the generated query vector
result = client.query(
    collection_name="summarizer_test",  # The collection name
    query_text=prompt,  # The query text
    query_vector=query_vector.tolist(),  # Pass the query vector
    limit=3,  # Limit the number of results
)

# Print the result
print(result)

TypeError: qdrant_client.qdrant_client.QdrantClient.search() got multiple values for keyword argument 'query_vector'

In [61]:
context = "\n".join(r.document for r in result)
context

'aapl-20240928\nfalse\n2024\nFY\n0000320193\nP1Y\nP1Y\nP1Y\nP1Y\nhttp://fasb.org/us-gaap/2024#MarketableSecuritiesCurrent http://fasb.org/us-gaap/2024#MarketableSecuritiesNoncurrent\nhttp://fasb.org/us-gaap/2024#MarketableSecuritiesCurrent http://fasb.org/us-gaap/2024#MarketableSecuritiesNoncurrent\nhttp://fasb.org/us-gaap/2024#LongTermDebtCurrent http://fasb.org/us-gaap/2024#LongTermDebtNoncurrent\nhttp://fasb.org/us-gaap/2024#LongTermDebtCurrent http://fasb.org/us-gaap/2024#LongTermDebtNoncurrent\nhttp://fasb.org/us-gaap/2024#OtherAssetsNoncurrent\nhttp://fasb.org/us-gaap/2024#OtherAssetsNoncurrent\nhttp://fasb.org/us-gaap/2024#PropertyPlantAndEquipmentNet\nhttp://fasb.org/us-gaap/2024#PropertyPlantAndEquipmentNet\nhttp://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent\nhttp://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent\nhttp://fasb.org/us-gaap/2024#OtherLiabilitiesNoncurrent\nhttp://fasb.org/us-gaap/2024#OtherLiabilitiesNoncurrent\nhttp://fasb.org/us-gaap/2024#OtherLiabilitiesCurren

In [63]:
metaprompt = f"""
You are a financial analyst. 
Answer the following question using the provided context.
If you cant find the answer, do not pretend you know it, but answer "I dont know".

Question: {prompt.strip()}

Context: {context.strip()}
Answer:
"""
print(metaprompt)


You are a financial analyst. 
Answer the following question using the provided context.
If you cant find the answer, do not pretend you know it, but answer "I dont know".

Question: "
what is the ticker symbol of the company?

Context: aapl-20240928
false
2024
FY
0000320193
P1Y
P1Y
P1Y
P1Y
http://fasb.org/us-gaap/2024#MarketableSecuritiesCurrent http://fasb.org/us-gaap/2024#MarketableSecuritiesNoncurrent
http://fasb.org/us-gaap/2024#MarketableSecuritiesCurrent http://fasb.org/us-gaap/2024#MarketableSecuritiesNoncurrent
http://fasb.org/us-gaap/2024#LongTermDebtCurrent http://fasb.org/us-gaap/2024#LongTermDebtNoncurrent
http://fasb.org/us-gaap/2024#LongTermDebtCurrent http://fasb.org/us-gaap/2024#LongTermDebtNoncurrent
http://fasb.org/us-gaap/2024#OtherAssetsNoncurrent
http://fasb.org/us-gaap/2024#OtherAssetsNoncurrent
http://fasb.org/us-gaap/2024#PropertyPlantAndEquipmentNet
http://fasb.org/us-gaap/2024#PropertyPlantAndEquipmentNet
http://fasb.org/us-gaap/2024#OtherLiabilitiesCurrent
h

In [64]:
from openai import AsyncOpenAI

client_gpt = AsyncOpenAI()
completion = await client_gpt.chat.completions.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": metaprompt}])
response = completion.choices[0].message.content
print(response)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 50438 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}